In [29]:
import numpy as np
from pathlib import Path
import re
import pandas as pd
import networkx as nx
from cloudvolume import CloudVolume, Skeleton
from io import StringIO
from brainlit.utils.session import read_s3, df_to_graph

s3_path = "s3://open-neurodata/brainlit/brain1_segments"
mip = 0

df_neuron = read_s3(s3_path, 120, mip)
g_neuron = df_to_graph(df_neuron)
print(df_neuron)
print(type(g_neuron))

Downloading: 100%|██████████| 1/1 [00:00<00:00, 18.91it/s]
     sample  structure        x        y       z    r  parent
0         1          0  14263.0  15815.0  2970.0  1.0      -1
1         2          0  14301.0  15837.0  2974.0  1.0       1
2        10          0  14363.0  15789.0  2975.0  1.0       2
3        12          0  14385.0  15709.0  2985.0  1.0      10
4        24          0  14445.0  15613.0  2989.0  1.0      12
..      ...        ...      ...      ...     ...  ...     ...
805     295          0  14981.0  14471.0  3014.0  1.0     353
806     307          0  14993.0  14369.0  3011.0  1.0     295
807     333          0  15013.0  14257.0  3032.0  1.0     307
808     337          0  15015.0  14198.0  3074.0  1.0     333
809     351          0  15034.0  14219.0  3087.0  1.0     337

[810 rows x 7 columns]
<class 'networkx.classes.digraph.DiGraph'>
